In [1]:
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
from astropy.table import Table, vstack
import astropy
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.time import Time
import getpass

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath
from lsst.daf.butler import Butler
import lsst.geom as geom

import lsst.daf.base as dafBase
import lsst.afw.table as afwTable

import lsst.sphgeom as sphgeom

from lsst.ctrl.mpexec import pipeline2dot
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [2]:
repo = '/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo'
collections = ["LSSTCam-imSim/defaults"]
butler = Butler(repo, collections=collections)

# refs = list(set(butler.registry.queryDatasets("raw", visit=230)))
# print(len(refs))

In [3]:
# refs[0].dataId

In [4]:
# raw_test = butler.get('raw', dataId=refs[0].dataId)

In [5]:
# raw_test

In [6]:
where = f"tract = {3838} AND patch = {36} AND " + \
              f"skymap = 'DC2' AND band='g'"
refs = list(set(butler.registry.queryDatasets("raw", where=where)))
print(len(refs))
# raw_test = butler.get('raw', dataId=refs[0].dataId)

422899


In [7]:
refs[0].dataId

{instrument: 'LSSTCam-imSim', detector: 184, exposure: 479255, band: 'g', day_obs: 20231130, group: '479255', physical_filter: 'g_sim_1.4'}

In [8]:
days_obs = np.unique([i.dataId["day_obs"] for i in refs])

In [9]:
days_obs

array([20211231, 20220201, 20220222, 20220730, 20220801, 20220804,
       20220825, 20220827, 20220831, 20220902, 20220916, 20220917,
       20220918, 20220919, 20220920, 20221001, 20221004, 20221014,
       20221017, 20221020, 20221022, 20221023, 20221026, 20221031,
       20221103, 20221112, 20221115, 20221118, 20221129, 20221202,
       20221210, 20221213, 20221214, 20221216, 20221217, 20221218,
       20221219, 20221228, 20221231, 20230113, 20230718, 20230719,
       20230820, 20230821, 20230904, 20230907, 20230908, 20230921,
       20230924, 20231005, 20231008, 20231009, 20231020, 20231023,
       20231111, 20231118, 20231121, 20231129, 20231130, 20231202,
       20231203, 20231205, 20231207, 20231208, 20231218, 20231221,
       20240709, 20240801, 20240807, 20240812, 20240901, 20240905,
       20240906, 20240913, 20240924, 20240927, 20241004, 20241009,
       20241012, 20241021, 20241022, 20241023, 20241024, 20241026,
       20241027, 20241104, 20241106, 20241109, 20241112, 20241

In [10]:
where = f"tract = {3838} AND patch = {36} AND " + \
              f"skymap = 'DC2' AND band='g' AND day_obs in ({days_obs[1]})"
refs = list(set(butler.registry.queryDatasets("raw", where=where)))
print(len(refs))

92


In [11]:
repo = '/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo'
collections = ['2.2i/calib', 'skymaps', 'LSSTCam-imSim/defaults']
butler = Butler(repo)

In [12]:
my_collection_identifier = 'test_4222025_2'
print(my_collection_identifier)

user = os.getenv("USER")
print(user)

my_outputCollection = f"u/{user}_pm/{my_collection_identifier}" 
print('Name of new butler collection for my output: ', my_outputCollection)

test_4222025_2
taceroc
Name of new butler collection for my output:  u/taceroc_pm/test_4222025_2


In [13]:
! echo $PSCRATCH

/pscratch/sd/t/taceroc


In [14]:
simpleButler = SimplePipelineExecutor.prep_butler(repo, 
                                                  inputs=collections, 
                                                  output=my_outputCollection)

In [15]:
simpleButler.registry.getCollectionChain(my_outputCollection)

('u/taceroc_pm/test_4222025_2/20250422T162657Z',
 '2.2i/calib',
 'skymaps',
 'LSSTCam-imSim/defaults')

In [16]:
yaml_file = '$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml'
# steps = 'step1,step2'
steps = 'step1,step2'
# steps = 'step1,step2,step3'
my_uri = yaml_file + '#' + steps
print(my_uri)

$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml#step1,step2


In [17]:
assembleCoaddPipeline = Pipeline.from_uri(my_uri)

In [18]:
my_tract = 3838
my_patch = 36

In [19]:
# refs = list(set(butler.registry.queryDatasets("raw", collections=collections)))

In [20]:
# queryString = f"tract = {my_tract} AND patch = {my_patch} AND " + \
#               f"skymap = 'DC2' AND band='g' AND day_obs in ({days_obs[1]})"
# print(queryString)

In [21]:
# refs = list(set(butler.registry.queryDatasets("dark", where=queryString)))

In [22]:
# len(refs)

In [23]:
# pipeline2dot(assembleCoaddPipeline, "/global/homes/t/taceroc/custom_coadd_qgraph.dot")

# ! dot -Tpng "/global/homes/t/taceroc/custom_coadd_qgraph.dot" > "/global/homes/t/taceroc/custom_coadd_qgraph.png"

In [24]:
queryString = f"tract = {my_tract} AND patch = {my_patch} AND " + \
              f"skymap = 'DC2' AND band='g'"
spe = SimplePipelineExecutor.from_pipeline(assembleCoaddPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)

In [25]:
# sorted(list(set(butler.registry.queryDatasets('raw', collections=['2.2i/calib', 'skymaps', 'LSSTCam-imSim/defaults']))))

In [26]:
# !butler remove-runs /global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo u/taceroc_pm/test_4222025_1

In [27]:
quanta = spe.run()

/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/meas_extensions_shapeHSM/g46d310ee33+21ad69e7e1/python/lsst/meas/extensions/shapeHSM/_hsm_higher_moments.py:211: RuntimeWarning: invalid value encountered in scalar divide
  results[(p, q)] = np.sum(std_x_powers[p] * std_y_powers[q] * image_weight) / normalization
/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/meas_extensions_shapeHSM/g46d310ee33+21ad69e7e1/python/lsst/meas/extensions/shapeHSM/_hsm_higher_moments.py:201: RuntimeWarning: invalid value encountered in scalar divide
  scale = image_array[~badpix].sum() / weight[~badpix].sum()


In [ ]:
# !butler remove-runs /global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo u/taceroc_pm/test_4222025_1/*

The following RUN collections will be removed:
u/taceroc_pm/test_4222025_1/20250422T125940Z
u/taceroc_pm/test_4222025_1/20250422T132604Z
u/taceroc_pm/test_4222025_1/20250422T134014Z
u/taceroc_pm/test_4222025_1/20250422T142052Z
u/taceroc_pm/test_4222025_1/20250422T160810Z: will be unlinked from "u/taceroc_pm/test_4222025_1"

The following datasets will be removed:
assembleCoadd_config(1), calibrate_config(1), characterizeImage_config(1), consolidateSourceTable_config(1), consolidateVisitSummary_config(1), deblend_config(1), deepCoadd_deblendedFlux_schema(1), deepCoadd_det_schema(1), deepCoadd_meas_schema(1), deepCoadd_mergeDet_schema(1), deepCoadd_peak_schema(1), detection_config(1), finalVisitSummary_schema(1), finalizeCharacterization_config(1), icSrc_schema(1), isolatedStarAssociation_config(1), isr_config(1), isr_log(17), isr_metadata(16), makeCcdVisitTable_config(1), makeVisitTable_config(1), makeWarp_config(1), measure_config(1), mergeDetections_config(1), packages(1), postISRCCD(